In [3]:
import numpy as np
k=4
obs_dim = 1
ctrl_dim = 1
state_dim = k*obs_dim + k*ctrl_dim - ctrl_dim

# First we construct the system matrices
A = np.zeros((state_dim, state_dim))
B = np.zeros((state_dim, ctrl_dim))

# Constant term
# A[-1,-1] = 1
# Shift history
n = obs_dim
l = ctrl_dim
m = obs_dim + ctrl_dim

if k > 1:
    A[0 : n, 0 : n] = np.eye(n)
    A[n : 2*n, 0 : n] = np.eye(n)
for i in range(k-2):
    A[(i+1)*m+n : (i+2)*m+n, i*m+n : (i+1)*m+n] = np.eye(m)

# Predict new observation
#A[0 : n, :] = coeffs[:, :-l]

# Add new control
# B[0 : n, :] = coeffs[:, -l:]
B[2*n : 2*n + l, :] = np.eye(l)

print(A, B)

[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]] [[0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [2]:
import autompc
import importlib
importlib.reload(autompc)
from autompc.sysid.mlp import ARMLP
from autompc.sysid import ARX
observations = ['current_exp0','current_exp1','current_exp2','current_exp3','current_exp4','current_exp5','current_exp6','current_exp7','current_exp8','current_exp9','pos_exp0','pos_exp1','pos_exp2','pos_exp3','pos_exp4']
controls = ['duty_exp0','duty_exp1','duty_exp2','duty_exp3','duty_exp4','duty_exp5','duty_exp6','duty_exp7','duty_exp8','duty_exp9']
observations = ['x']
controls = ['u']
system = autompc.System(observations, controls, dt=0.02)

model = ARMLP(system, history=4)
ARXmodel = ARX(system)
print(model.state_system.observations)
print(ARXmodel.state_system.observations)
print(ARXmodel.A)

# 1 0 0 0 0 0 0
# 1 0 0 0 0 0 0
# 0 0 0 0 0 0 1
# 0 1 0 0 0 0 0
# 0 0 1 0 0 0 0
# 0 0 0 1 0 0 0
# 0 0 0 0 0 0 0 k


Loading AutoMPC...
Finished loading AutoMPC
use_cuda=True
torch.cuda.is_available()=True
self._device=device(type='cuda')
['x[t]', 'x[t-1]', 'u[t-1]', 'x[t-2]', 'u[t-2]', 'x[t-3]', 'u[t]']
['x[t]', 'x[t-1]', 'u[t-1]', 'x[t-2]', 'u[t-2]', 'x[t-3]', 'const', 'u[t]']


AttributeError: 'ARX' object has no attribute 'A'

In [ ]:
observations = ['current_exp0','current_exp1','current_exp2','current_exp3','current_exp4','current_exp5','current_exp6','current_exp7','current_exp8','current_exp9','pos_exp0','pos_exp1','pos_exp2','pos_exp3','pos_exp4']
controls = ['duty_exp0','duty_exp1','duty_exp2','duty_exp3','duty_exp4','duty_exp5','duty_exp6','duty_exp7','duty_exp8','duty_exp9']
system = ampc.System(observations, controls, dt=0.05)

for csv in glob('/home/dohun/Repos/spine_robot_autompc/data/real_data/*.csv'):
    
    traj = data_utils.csv_to_traj(system, csv)
    if csv.split('/')[-1][:-4] == 'freq1_duty123_shift3':
        # Cut at 6s and 11s for duty123shift3
        start = 0
        for cut in [6, 11]:
            end = np.searchsorted(traj.times, cut)
            trajs.append(traj[start:end])
            start = end
        trajs.append(traj[start:])
    elif csv.split('/')[-1][:-4] == 'freq2_duty123_shift123':
        # Cut at 8s, 12s, 15s, 25s, 28s, 32s, 38s, 43s
        start = 0
        for cut in [8, 12, 15, 25, 28, 32, 38, 43]:
            end = np.searchsorted(traj.times, cut)
            trajs.append(traj[start:end])
            start = end
        trajs.append(traj[start:])
    else:
        trajs.append(traj)
for traj in trajs:
    print(len(traj))

from autompc.sysid.mlp import ARMLP
model = ARMLP(system, history=3)
tuner = ModelTuner(system,trajs,model,verbose=1)
surrogate,tune_result = tuner.run(n_iters=100, eval_timeout=500)

print("Selected model:",surrogate.name)
print("Final cross-validated RMSE score:",tune_result.inc_costs[-1])